# Get new parametersets with Baysian Optimization

This script uses the BaysianBackEnd (BayBE). For more information you can find the documentation here: https://emdgroup.github.io/baybe/stable/index.html 

In [1]:
%%capture
!pip install baybe
from baybe.targets import NumericalTarget
from baybe.objectives.single import SingleTargetObjective
from baybe.constraints import ContinuousLinearInequalityConstraint

import pandas as pd
import numpy as np
import datetime

import ipywidgets as widgets
from IPython.display import display, clear_output

In [16]:
parameters_data_slow_1 = [
    ("BiNO3_M",0.005,0.05,0.005),
    ("KI_M",0.1,1,0.1),
    ("LA_M",0.01,0.06,0.01),
    ("benzoquinone_M",0.03,0.06,0.002),
    ("Vacac2_M",0.02,0.8,0.005),
    ("Ni_M",0.005,0.095,0.005),
]

# electrodeposition BiVO4
parameters_data_fast_1 = [
    ("N1_mAcm_2",-15,-1,1),
    ("G1_mAcm_2",-10,-0.5,0.1),
    ("N1_s",5,45,5),
    ("G1_s",10,70,10),
    ("NG1_s",10,80,10),
    ("no_cycles",10,105,10),
]

# synthesis TiO2
parameters_data_slow_2 = [
    ("TiCl3_M",0.02,0.1,0.01),
    ("Na2CO3_M",0.1,1,0.2),
    ("pH",0.5,7,0.5),
    ("Temperature_C",25,90,5),
]

# electrodeposition TiO2 (maybe N, G, NG parameters missing?)
parameters_data_fast_2 = [
    ("potential_V",0,2,0.2),
    ("N_s",10,60,0.5),
]
constraints = None

def get_parameters_data(parameter_set):
    params_data = []
    if parameter_set == 'synthesis BiVO4 and NiFe ("slow")':
        params_data = parameters_data_slow_1
    elif parameter_set == 'BiVO4 electrodeposition ("fast")':
        params_data = parameters_data_fast_1
    elif parameter_set == "slow_2":
        params_data = parameters_data_slow_2
    elif parameter_set == "fast_2":
        params_data = parameters_data_fast_2
    return params_data

In [17]:
# all ipywidgets
parameter_selector = widgets.Dropdown(
    options=['BiVO4 electrodeposition ("fast")', 'synthesis BiVO4 and NiFe ("slow")'],
    value='BiVO4 electrodeposition ("fast")',
    description='Select parameter set:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px'),
)

author_selector = widgets.Dropdown(
    options=['Felipe Mata mata', 'Maddalena Zoli'],
    value=None,
    description='NOMAD author:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px'),
)

run_baybe_button = widgets.Button(
    description='Run Bayesian Optimization',
    button_style='primary',
    layout=widgets.Layout(width='auto')
)

save_recommendations_button = widgets.Button(
    description='Save recommended parameters as csv',
    button_style='primary',
    layout=widgets.Layout(width='auto')
)

author_measurement_overview_output = widgets.Output()
recommendations_output = widgets.Output()
save_csv_output = widgets.Output()

In [18]:
def update_measurement_overview_output():
    global parameters_data, measurements
    parameters_data = get_parameters_data(parameter_selector.value)
    first_name = author_selector.value.split()[0].lower()
    param_name = 'bivo4_synthesis'
    if parameter_selector.value == 'BiVO4 electrodeposition ("fast")':
        param_name = 'bivo4_electrodeposition'
    file_name = f'baybe_csv/{param_name}_{first_name}.csv'
    with author_measurement_overview_output:
        author_measurement_overview_output.clear_output()
        try:
            measurements = pd.read_csv(file_name)
            print('Measurements for given user:')
            display(measurements)
        except FileNotFoundError:
            print(f'Could not find file {file_name}')

def on_paramset_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_measurement_overview_output()

parameter_selector.observe(on_paramset_change)
parameter_selector.value = 'BiVO4 electrodeposition ("fast")'

def on_author_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_measurement_overview_output()
            

author_selector.observe(on_author_change)
author_selector.value = 'Felipe Mata mata'

display(parameter_selector, author_selector, author_measurement_overview_output)

Dropdown(description='Select parameter set:', layout=Layout(width='500px'), options=('BiVO4 electrodeposition …

Dropdown(description='NOMAD author:', layout=Layout(width='500px'), options=('Felipe Mata mata', 'Maddalena Zo…

Output()

In [19]:
def scale_measurements_down(df, parameter_info):
    for p in parameter_info:
        values = (df[p[0]]-p[1])/(p[2]-p[1])
        df[p[0]] = values
    return df

def scale_parameters_up(df, parameter_info):
    for p in parameter_info:
        values = (p[2]-p[1])*df[p[0]] + p[1]
        df[p[0]] = round(values / p[3]) * p[3]
    return df

In [20]:
from baybe.parameters import NumericalContinuousParameter
from baybe.searchspace import SearchSpace
from baybe.recommenders import (
    BotorchRecommender,
    RandomRecommender,
    TwoPhaseMetaRecommender,
)
from baybe import Campaign


def get_clean_baybe_setup(parameters_data):
    constraints = None
    parameters = [
        NumericalContinuousParameter(
            name=p[0],
             bounds=(0, 1),      
        ) for p in parameters_data
    ]
    recommender = TwoPhaseMetaRecommender(
        initial_recommender=RandomRecommender(),  # farthest point sampling
        recommender=BotorchRecommender(sequential_continuous=True),  # Bayesian model-based optimization
    )
    searchspace = SearchSpace.from_product(parameters,constraints=constraints)

    #print('Maximize for target activity.')
    #target = NumericalTarget(name="activity", mode="MAX")
    print('Minimize for target stability.')
    target = NumericalTarget(name="stability", mode="MIN")
    objective = SingleTargetObjective(target)
    columns = [p[0] for p in parameters_data]
    campaign = Campaign(searchspace, objective, recommender)
    return campaign

def get_baybe_recommendations(campaign, parameters_data, measurements, num_recommendations=5):
    scaled_measurements = measurements.copy() # use a copy if user hits recommendation button several times
    scaled_measurements = scale_measurements_down(scaled_measurements, parameters_data)
    # print(scaled_measurements) # uncomment to check if measurements are scaled down as expected
    campaign.add_measurements(scaled_measurements)
    df_next = campaign.recommend(batch_size=num_recommendations)
    # print(df_next) #uncomment to see unscaled recommendations
    df_next = scale_parameters_up(df_next, parameters_data)
    return df_next

def save_recommendations_as_csv(df_next, author_name):
    date_now = datetime.datetime.now()
    filename = f'baybe_csv/newparameters_ni_{author_name}_{date_now.strftime("%Y%m%d")}.csv'
    df_next.to_csv(filename, index=False, header=True)
    print(f'File saved in {filename}')

In [21]:
def on_baybe_button_clicked(b):
    global df_pretty
    with recommendations_output:
        recommendations_output.clear_output()
        campaign = get_clean_baybe_setup(parameters_data)
        df_pretty = get_baybe_recommendations(campaign, parameters_data, measurements, num_recommendations=6)
        print('Recommended next parameter sets:')
        display(df_pretty)

run_baybe_button.on_click(on_baybe_button_clicked)

display(run_baybe_button, recommendations_output)

Button(button_style='primary', description='Run Bayesian Optimization', layout=Layout(width='auto'), style=But…

Output()

In [8]:
def on_save_csv_button_clicked(b):
    with save_csv_output:
        save_csv_output.clear_output()
        first_name = author_selector.value.split()[0].lower()
        save_recommendations_as_csv(df_pretty, first_name)

save_recommendations_button.on_click(on_save_csv_button_clicked)

display(save_recommendations_button, save_csv_output)

Button(button_style='primary', description='Save recommended parameters as csv', layout=Layout(width='auto'), …

Output()